In [ ]:
!nvidia-smi

Sat Sep 11 09:17:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    50W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers SentencePiece torch tqdm

In [ ]:
import math

from tqdm import tqdm
import numpy as np
from transformers import MT5ForConditionalGeneration, T5Tokenizer
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score

In [ ]:
class SoftEmbedding(nn.Module):
    def __init__(self, 
                wte: nn.Embedding,
                n_tokens: int = 10, 
                random_range: float = 0.5,
                initialize_from_vocab: bool = True):
        """appends learned embedding to 
        Args:
            wte (nn.Embedding): original transformer word embedding
            n_tokens (int, optional): number of tokens for task. Defaults to 10.
            random_range (float, optional): range to init embedding (if not initialize from vocab). Defaults to 0.5.
            initialize_from_vocab (bool, optional): initalizes from default vocab. Defaults to True.
        """
        super(SoftEmbedding, self).__init__()
        self.wte = wte
        self.n_tokens = n_tokens
        self.learned_embedding = nn.parameter.Parameter(self.initialize_embedding(wte,
                                                                                  n_tokens, 
                                                                                  random_range, 
                                                                                  initialize_from_vocab))
            
    def initialize_embedding(self, 
                             wte: nn.Embedding,
                             n_tokens: int = 10, 
                             random_range: float = 0.5, 
                             initialize_from_vocab: bool = True):
        """initializes learned embedding
        Args:
            same as __init__
        Returns:
            torch.float: initialized using original schemes
        """
        if initialize_from_vocab:
            return self.wte.weight[:n_tokens].clone().detach()
        return torch.FloatTensor(n_tokens, wte.weight.size(1)).uniform_(-random_range, random_range)
            
    def forward(self, tokens):
        """run forward pass
        Args:
            tokens (torch.long): input tokens before encoding
        Returns:
            torch.float: encoding of text concatenated with learned task specifc embedding
        """
        input_embedding = self.wte(tokens[:, self.n_tokens:])
        learned_embedding = self.learned_embedding.repeat(input_embedding.size(0), 1, 1)
        return torch.cat([learned_embedding, input_embedding], 1)

In [ ]:
!pip install zh-dataset-inews

In [ ]:
from zh_dataset_inews import title_train, label_train, title_dev, label_dev, title_test, label_test

In [ ]:
def generate_data(batch_size, n_tokens, title_data, label_data):

    labels = [
        torch.tensor([[3]]),  # \x00
        torch.tensor([[4]]),  # \x01
        torch.tensor([[5]]),  # \x02
    ]

    def yield_data(x_batch, y_batch, l_batch):
        x = torch.nn.utils.rnn.pad_sequence(x_batch, batch_first=True)
        y = torch.cat(y_batch, dim=0)
        m = (x > 0).to(torch.float32)
        decoder_input_ids = torch.full((x.size(0), n_tokens), 1)
        if torch.cuda.is_available():
            x = x.cuda()
            y = y.cuda()
            m = m.cuda()
            decoder_input_ids = decoder_input_ids.cuda()
        return x, y, m, decoder_input_ids, l_batch

    x_batch, y_batch, l_batch = [], [], []
    for x, y in zip(title_data, label_data):
        context = x
        inputs = tokenizer(context, return_tensors="pt")
        inputs['input_ids'] = torch.cat([torch.full((1, n_tokens), 1), inputs['input_ids']], 1)
        l_batch.append(y)
        y = labels[y]
        y = torch.cat([torch.full((1, n_tokens - 1), -100), y], 1)
        x_batch.append(inputs['input_ids'][0])
        y_batch.append(y)
        if len(x_batch) >= batch_size:
            yield yield_data(x_batch, y_batch, l_batch)
            x_batch, y_batch, l_batch = [], [], []

    if len(x_batch) > 0:
        yield yield_data(x_batch, y_batch, l_batch)
        x_batch, y_batch, l_batch = [], [], []

In [ ]:
model = MT5ForConditionalGeneration.from_pretrained("google/mt5-large")
tokenizer = T5Tokenizer.from_pretrained("google/mt5-large")
n_tokens = 100
s_wte = SoftEmbedding(model.get_input_embeddings(), 
                      n_tokens=n_tokens, 
                      initialize_from_vocab=True)
model.set_input_embeddings(s_wte)
if torch.cuda.is_available():
    model = model.cuda()

In [ ]:
parameters = list(model.parameters())
for x in parameters[1:]:
    x.requires_grad = False

In [ ]:
parameters[0]

Parameter containing:
tensor([[ -1.0312,  -4.2500,   7.0000,  ...,   6.0938,  -8.0625,  -9.5000],
        [ -7.7500, -12.1250,  -2.3438,  ...,  -7.8438,   9.1875,   4.4375],
        [  0.9805,   1.0781,  -0.3867,  ...,  -1.0156,  -0.4785,   0.8008],
        ...,
        [ -1.4922,   0.1895,  -0.2041,  ...,   0.6250,   0.0131,  -1.8828],
        [  0.8789,   0.1108,   1.1953,  ...,   0.8281,   1.4844,   0.3418],
        [  0.1436,  -0.3867,  -0.7734,  ...,   0.5078,  -0.0157,   0.1060]],
       device='cuda:0', requires_grad=True)

In [ ]:
parameters[2]

Parameter containing:
tensor([[ 0.0099,  0.0084,  0.0172,  ...,  0.0220,  0.0435, -0.0337],
        [ 0.0112, -0.0181, -0.0107,  ...,  0.0227,  0.0190,  0.0033],
        [ 0.0061,  0.0430,  0.0625,  ..., -0.0334, -0.0130,  0.0205],
        ...,
        [ 0.0034,  0.0228,  0.0003,  ...,  0.0113, -0.0045, -0.0222],
        [ 0.0297, -0.0042, -0.0393,  ...,  0.0037, -0.0145, -0.0023],
        [ 0.0053, -0.0029,  0.0157,  ..., -0.0125,  0.0068,  0.0106]],
       device='cuda:0')

In [ ]:
for x, y, m, dii, true_labels in generate_data(2, n_tokens, title_train, label_train):
    assert dii.shape == y.shape
    outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
    assert outputs['logits'].shape[:2] == y.shape
    pred_labels = outputs['logits'][:, -1, 3:6].argmax(-1).detach().cpu().numpy().tolist()
    break

In [13]:
batch_size = 2
n_epoch = 50
total_batch = math.ceil(len(title_train) / batch_size)
dev_total_batch = math.ceil(len(title_dev) / batch_size)
use_ce_loss = False
ce_loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(s_wte.parameters(), lr=0.5)

for epoch in range(n_epoch):
    print('epoch', epoch)

    all_true_labels = []
    all_pred_labels = []
    losses = []
    pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, title_train, label_train)), total=total_batch)
    for i, (x, y, m, dii, true_labels) in pbar:
        all_true_labels += true_labels
        
        optimizer.zero_grad()
        outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
        pred_labels = outputs['logits'][:, -1, 3:6].argmax(-1).detach().cpu().numpy().tolist()
        all_pred_labels += pred_labels

        if use_ce_loss:
            logits = outputs['logits'][:, -1, 3:6]
            true_labels_tensor = torch.tensor(true_labels, dtype=torch.long).cuda()
            loss = ce_loss(logits, true_labels_tensor)
        else:
            loss = outputs.loss
        loss.backward()
        optimizer.step()
        loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
        losses.append(loss_value)

        acc = accuracy_score(all_true_labels, all_pred_labels)
        pbar.set_description(f'train: loss={np.mean(losses):.4f}, acc={acc:.4f}')

    all_true_labels = []
    all_pred_labels = []
    losses = []
    with torch.no_grad():
        pbar = tqdm(enumerate(generate_data(batch_size, n_tokens, title_dev, label_dev)), total=dev_total_batch)
        for i, (x, y, m, dii, true_labels) in pbar:
            all_true_labels += true_labels
            outputs = model(input_ids=x, labels=y, attention_mask=m, decoder_input_ids=dii)
            loss = outputs.loss
            loss_value = float(loss.detach().cpu().numpy().tolist()) / batch_size
            losses.append(loss_value)
            pred_labels = outputs['logits'][:, -1, 3:6].argmax(-1).detach().cpu().numpy().tolist()
            all_pred_labels += pred_labels
            acc = accuracy_score(all_true_labels, all_pred_labels)
            pbar.set_description(f'dev: loss={np.mean(losses):.4f}, acc={acc:.4f}')

epoch 0


dev: loss=24.9345, acc=0.0861: 100%|██████████| 500/500 [00:32<00:00, 15.35it/s]


epoch 1


dev: loss=16.0809, acc=0.4344: 100%|██████████| 500/500 [00:32<00:00, 15.47it/s]


epoch 2


...

dev: loss=0.3732, acc=0.7477: 100%|██████████| 500/500 [00:32<00:00, 15.37it/s]


epoch 48


dev: loss=0.4366, acc=0.7227: 100%|██████████| 500/500 [00:32<00:00, 15.33it/s]


epoch 49


dev: loss=0.4849, acc=0.7387: 100%|██████████| 500/500 [00:32<00:00, 15.41it/s]


In [14]:
parameters2 = list(model.parameters())

In [15]:
parameters2[0]

Parameter containing:
tensor([[  39.9078, -138.6385,  217.1636,  ...,   29.5207,  145.8943,
          144.5315],
        [ 515.6390,  -31.6162,   51.5134,  ...,  -53.0618,  245.4292,
          -69.3007],
        [ 236.2896,   43.0374,  -19.2581,  ..., -127.3152,  130.7397,
           31.1689],
        ...,
        [  88.1486,   49.1501,  125.5696,  ...,  113.4881,   96.0846,
          368.0652],
        [ 100.6963, -102.7619,  -35.8637,  ..., -144.5385,  -25.3403,
          173.1718],
        [-164.1508,  -81.5056,  152.1980,  ..., -178.5098,    6.0514,
         -129.9609]], device='cuda:0', requires_grad=True)

In [16]:
parameters2[2]

Parameter containing:
tensor([[ 0.0099,  0.0084,  0.0172,  ...,  0.0220,  0.0435, -0.0337],
        [ 0.0112, -0.0181, -0.0107,  ...,  0.0227,  0.0190,  0.0033],
        [ 0.0061,  0.0430,  0.0625,  ..., -0.0334, -0.0130,  0.0205],
        ...,
        [ 0.0034,  0.0228,  0.0003,  ...,  0.0113, -0.0045, -0.0222],
        [ 0.0297, -0.0042, -0.0393,  ...,  0.0037, -0.0145, -0.0023],
        [ 0.0053, -0.0029,  0.0157,  ..., -0.0125,  0.0068,  0.0106]],
       device='cuda:0')

In [17]:
def predict(text):
    inputs = tokenizer(text, return_tensors='pt')
    inputs['input_ids'] = torch.cat([torch.full((1, n_tokens), 1), inputs['input_ids']], 1)

    decoder_input_ids = torch.full((1, n_tokens), 1)
    with torch.no_grad():
        outputs = model(input_ids=inputs['input_ids'].cuda(), decoder_input_ids=decoder_input_ids.cuda())
    logits = outputs['logits'][:, -1, 3:6]
    pred = logits.argmax(-1).detach().cpu().numpy()[0]
    # print(logits)
    return pred

In [18]:
train_rets = []
for i in tqdm(range(len(title_train))):
    pred = predict(title_train[i])
    train_rets.append((label_train[i], pred, title_train[i]))

100%|██████████| 5355/5355 [04:39<00:00, 19.19it/s]


In [ ]:
rets = []
for i in tqdm(range(len(title_test))):
    pred = predict(title_test[i])
    rets.append((label_test[i], pred, title_test[i]))

In [23]:
print(
    accuracy_score(
        [x[0] for x in train_rets],
        [x[1] for x in train_rets],
    )
)

0.861624649859944


In [24]:
print(
    accuracy_score(
        [x[0] for x in rets],
        [x[1] for x in rets],
    )
)

0.7447447447447447


In [25]:
print(
    accuracy_score(
        [x[0] for x in rets],
        [0] * len(rets),
    ),
    accuracy_score(
        [x[0] for x in rets],
        [1] * len(rets),
    ),
    accuracy_score(
        [x[0] for x in rets],
        [2] * len(rets),
    )
)

0.0990990990990991 0.4944944944944945 0.4064064064064064
